In [ ]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 11.9 MB/s eta 0:00:00


In [ ]:
!pip install PaDEL-pywrapper

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 8.3 MB/s eta 0:00:00
  Created wheel for install-jdk: filename=install_jdk-0.3.0-py3-none-any.whl size=3725 sha256=ae2b96e2820d9379ec124fd6fb19e61f8a72004c4db883424f09a5d90407cb8b
  Stored in directory: /root/.cache/pip/wheels/79/7a/47/9a4619174f7ca0f1068edb7a5412730a37365b6d183b0b3847
Successfully built install-jdk


In [ ]:
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski
from PaDEL_pywrapper import PaDEL
from PaDEL_pywrapper.descriptor import ALOGP, Crippen, FMF
from PaDEL_pywrapper import descriptors

ModuleNotFoundError: No module named 'rdkit'

In [ ]:
import pandas as pd
import numpy as np

## Loading collected data

In [ ]:
!wget https://raw.githubusercontent.com/cpariona/biomedical-thesis/refs/heads/main/data/docking_results/docking_scores_data.csv

--2024-12-09 03:18:40--  https://raw.githubusercontent.com/cpariona/biomedical-thesis/refs/heads/main/data/docking_results/docking_scores_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 338231 (330K) [text/plain]
Saving to: ‘docking_scores_data.csv’

docking_scores_data 100%[===================>] 330.30K  --.-KB/s    in 0.06s   

2024-12-09 03:18:40 (5.53 MB/s) - ‘docking_scores_data.csv’ saved [338231/338231]



In [ ]:
df = pd.read_csv("docking_scores_data.csv")
df.columns

Index(['smiles', 'Delta_Gibbs'], dtype='object')

In [ ]:
df.shape

(8530, 2)

In [ ]:
# Handling missing data && cleaning
df = df.dropna(subset=['Delta_Gibbs'])
df = df.drop_duplicates(['smiles']).reset_index(drop=True)

In [ ]:
df.shape

(8515, 2)

## Compute Molecular Descriptors

### Compute the RDKit discriptor

In [ ]:
def getRDKitDescriptors(smile, missingVal=None):
    ''' calculate the full list of descriptors for a molecule

        missingVal is used if the descriptor cannot be calculated
    '''
    res = {}
    for nm,fn in Descriptors._descList:
        # some of the descriptor fucntions can throw errors if they fail, catch those here:
        try:
            val = fn(Chem.MolFromSmiles(smile))
        except:
            # print the error message:
            import traceback
            traceback.print_exc()
            # and set the descriptor value to whatever missingVal is
            val = missingVal
        res[nm] = val
    return res

In [ ]:
# Calculate descriptors
RDKit_descriptors = [getRDKitDescriptors(m) for m in df['smiles'].tolist()]

In [ ]:
# Create RDKit DataFrame
RDKit_df = pd.DataFrame(RDKit_descriptors)

In [ ]:
RDKit_df.shape

(8515, 210)

In [ ]:
# Save RDKit descriptors
RDKit_df.to_csv('RDKit_descriptors.csv', index=False)

### Compute the PaDEL discriptor

In [ ]:
# Calculate descriptors
mols = [Chem.MolFromSmiles(smiles) for smiles in df['smiles'].tolist()]
padel = PaDEL(descriptors)
PaDEL_descriptors = padel.calculate(mols)

In [ ]:
PaDEL_descriptors.shape

(8515, 1444)

In [ ]:
# Save RDKit descriptors
PaDEL_descriptors.to_csv('PaDEL_descriptors.csv', index=False)